In [21]:
!pip install tabula-py
import tabula
import pandas as pd
import numpy as np

In [22]:
table = tabula.read_pdf("G1.pdf", pages=1)

In [23]:
for i in range(len(table)):
    table[i].columns = table[i].columns.str.replace(r"\r", " ")

In [24]:
table[0] = table[0].drop(columns=["Unnamed: 0"])

In [25]:
index = table[0].index
condition1 = table[0]['CHICOPEE BIG Y'] == "WEEKDAY"
condition2 = table[0]['CHICOPEE BIG Y'] == 'SATURDAY'
condition3 = table[0]['CHICOPEE BIG Y'] == 'SUNDAY'
day_indices = index[condition1][0], index[condition2][0], index[condition3][0]

table_weekday = table[0].iloc[day_indices[0] + 1: day_indices[1] - 1]
table_saturday = table[0].iloc[day_indices[1] + 1: day_indices[2] - 1]
table_sunday = table[0].iloc[day_indices[2] + 1: len(table[0])]
table_weekday.reset_index(drop=True, inplace=True)
table_saturday.reset_index(drop=True, inplace=True)
table_sunday.reset_index(drop=True, inplace=True)

In [26]:
def preprocess_table(table):
    for i in range(len(table.columns)):
        col = table.columns[i]
        table[col] = table[col].str.replace('[\|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|-]','')

In [27]:
preprocess_table(table_weekday)
preprocess_table(table_saturday)
preprocess_table(table_sunday)

C:\Users\quynh\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
# condition on mornings for weekdays. 
condition_weekday_morning = table_weekday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_morning = table_weekday.iloc[0: condition_weekday_morning]
table_weekday_morning.reset_index(drop=True, inplace=True)

In [29]:
# condition on evenings for weekdays
condition_weekday_evening = table_weekday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_evening = table_weekday.iloc[condition_weekday_evening - 1:]
table_weekday_evening.reset_index(drop=True, inplace=True)

In [30]:
# condition on mornings for saturdays. 
condition_saturday_morning = table_saturday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_morning = table_saturday.iloc[0: condition_saturday_morning]
table_saturday_morning.reset_index(drop=True, inplace=True)

In [31]:
# condition on evenings for saturdays. 
condition_saturday_evening = table_saturday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_evening = table_saturday.iloc[condition_saturday_evening - 1:]
table_saturday_evening.reset_index(drop=True, inplace=True)

In [32]:
# condition on mornings for sunday. 
condition_sunday_morning = table_sunday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_morning = table_sunday.iloc[0: condition_sunday_morning]
table_sunday_morning.reset_index(drop=True, inplace=True)

In [33]:
# condition on evenings for sunday. 
condition_sunday_evening = table_sunday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_evening = table_sunday.iloc[condition_sunday_evening - 1:]
table_sunday_evening.reset_index(drop=True, inplace=True)

In [34]:
def preprocess_times_morning(table):
    for i in range(len(table.columns)):
        if i != 0:
            col = table.columns[i]
            index = table[col].str.contains('[1][2]:[0-5][0-9]').idxmax()
            table.loc[index:,col] = ''

In [35]:
def preprocess_times_evening(table):
    for i in range(len(table.columns)):
        col = table.columns[i]
        index = table[col].str.contains('[1][2]:[0-5][0-9]').idxmax()
        if index != -1:
            table.loc[0:index-1,col] = ''

In [36]:
table_weekday_evening = table_weekday_evening.copy()
preprocess_times_evening(table_weekday_evening)
table_saturday_evening = table_saturday_evening.copy()
preprocess_times_evening(table_saturday_evening)
table_sunday_evening = table_sunday_evening.copy()
preprocess_times_evening(table_sunday_evening)
table_weekday_morning = table_weekday_morning.copy()
preprocess_times_morning(table_weekday_morning)
table_saturday_morning = table_saturday_morning.copy()
preprocess_times_morning(table_saturday_morning)
table_sunday_morning = table_sunday_morning.copy()
preprocess_times_morning(table_sunday_morning)

In [37]:
table_weekday_morning = table_weekday_morning.replace(r'^\s*$', np.nan, regex=True)
table_weekday_evening = table_weekday_evening.replace(r'^\s*$', np.nan, regex=True)
table_saturday_morning = table_saturday_morning.replace(r'^\s*$', np.nan, regex=True)
table_saturday_evening = table_saturday_evening.replace(r'^\s*$', np.nan, regex=True)
table_sunday_morning = table_sunday_morning.replace(r'^\s*$', np.nan, regex=True)
table_sunday_evening = table_sunday_evening.replace(r'^\s*$', np.nan, regex=True)

# Drop apply
table_weekday_morning = table_weekday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_weekday_evening = table_weekday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_saturday_morning = table_saturday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_saturday_evening = table_saturday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_sunday_morning = table_sunday_morning.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
table_sunday_evening = table_sunday_evening.apply(lambda x: pd.Series(x.dropna().values)).fillna('')

In [38]:
table_weekday_morning

,CHICOPEE BIG Y,CHICOPEE CENTER,UNION STATION BAY 2,"THE ""X""",RIVERBEND MEDICAL CENTER,ALLEN & COOLEY/ 5 TOWN PLAZA,CANON CIRCLE,ALLEN & COOLEY/ 5 TOWN PLAZA.1,"THE ""X"".1",UNION STATION BAY 6,CHICOPEE CENTER.1,CHICOPEE BIG Y.1
0,6:30,6:25,6:00,6:10,6:50,6:55,6:30,5:35,5:45,6:10,6:00,6:18
1,6:45,6:40,6:15,6:25,7:45,7:55,7:00,6:35,6:45,6:25,6:25,6:43
2,7:05,7:00,6:25,6:35,8:25,8:35,7:15,7:20,6:30,7:15,6:45,7:03
3,7:25,7:20,6:45,6:55,8:45,8:55,7:35,7:40,7:10,6:55,7:35,7:58
4,7:45,7:40,7:00,7:15,9:25,9:35,8:00,8:20,7:30,7:40,7:15,7:38
5,8:05,8:00,7:20,7:35,10:05,10:15,8:15,9:20,7:50,8:00,8:00,8:23
6,8:25,8:20,7:40,7:55,10:45,10:55,8:40,10:00,8:10,8:20,8:20,8:43
7,8:45,8:40,8:00,8:15,11:25,11:35,9:00,10:40,8:30,8:40,8:35,9:18
8,9:25,9:00,8:20,8:35,,,9:15,11:20,8:50,9:00,8:55,9:58
9,10:05,9:20,8:40,8:55,,,9:40,,9:10,9:20,9:15,10:38


In [20]:
table_weekday_morning.to_csv(r'./PVTA_Route_Data/weekday_morning.csv', index=False, header=True)
table_weekday_evening.to_csv(r'./PVTA_Route_Data/weekday_evening.csv', index=False, header=True)
table_saturday_morning.to_csv(r'./PVTA_Route_Data/saturday_morning.csv', index=False, header=True)
table_saturday_evening.to_csv(r'./PVTA_Route_Data/saturday_evening.csv', index=False, header=True)
table_sunday_morning.to_csv(r'./PVTA_Route_Data/sunday_morning.csv', index=False, header=True)
table_sunday_evening.to_csv(r'./PVTA_Route_Data/sunday_evening.csv', index=False, header=True)